In [1]:
!pip install surprise

In [2]:
import io, os, json
from surprise import Dataset ,Reader, accuracy, SVD, dump
import pandas as pd
import numpy as np
from itertools import chain

In [3]:
song_meta = pd.read_pickle("/content/drive/My Drive/recommendation sys/song_meta_0802.pkl") # new_tags와 cluster_id가 들어있는 song_meta

In [6]:
# id_to_item dictionary

tags = pd.unique(list(chain.from_iterable(song_meta.new_tags)))
sid_to_cid = dict(zip(song_meta.id.map(str), song_meta.new_song_id.map(str)))

n_clusters = 1000
n_tags = 110

tag_to_tid = {tags[i]:str(n_clusters + i) for i in range(len(tags))}
tid_to_tag = {tid:tag for tag,tid in tag_to_tid.items()}
sid_to_song = song_meta.song_name.to_dict()

In [7]:
# validation set
# 속한 노래와 태그를 id로 mapping한 후 합쳐서 cluster_tags에 list형태로 저장

val = pd.read_json("/content/drive/My Drive/recommendation sys/0802/val.json")
val['cid'] = val.songs.map(lambda x:[sid_to_cid[str(i)] for i in x])
val['tid'] = val.tags.map(lambda x:[tag_to_tid[i] for i in x if i in tags])
val['clusters_tags'] = val.cid + val.tid
val

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,"[673, 673, 55, 833, 776, 424, 472, 666, 234, 7...",[],"[673, 673, 55, 833, 776, 424, 472, 666, 234, 7..."
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000,[],[],[]
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000,"[166, 722, 285, 285, 290, 937, 910, 220, 726, ...",[],"[166, 722, 285, 285, 290, 937, 910, 220, 726, ..."
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000,"[655, 493, 213, 956, 145, 510, 145, 485, 857, ...",[],"[655, 493, 213, 956, 145, 510, 145, 485, 857, ..."
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000,"[605, 940, 764, 764, 751, 584, 940, 584]",[],"[605, 940, 764, 764, 751, 584, 940, 584]"
...,...,...,...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000,"[804, 194, 935, 87, 655, 133, 857, 803, 523, 9...",[1036],"[804, 194, 935, 87, 655, 133, 857, 803, 523, 9..."
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000,"[377, 834, 418, 613, 295, 152, 787, 489, 286, ...",[],"[377, 834, 418, 613, 295, 152, 787, 489, 286, ..."
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000,"[857, 857, 857, 857, 857, 857, 857, 857, 857, ...",[],"[857, 857, 857, 857, 857, 857, 857, 857, 857, ..."
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000,"[531, 615, 155, 347, 70, 347, 347, 615, 618]",[],"[531, 615, 155, 347, 70, 347, 347, 615, 618]"


In [8]:
tr = pd.read_json("/content/drive/My Drive/recommendation sys/0802/train.json")
tr['cid'] = tr.songs.map(lambda x:[sid_to_cid[str(i)] for i in x])
tr['tid'] = tr.tags.map(lambda x:[tag_to_tid[str(i)] for i in x if x in tags])
tr['clusters_tags'] = tr.cid + tr.tid
tr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[509, 10, 783, 550, 346, 346, 604, 745, 357, 7...",[1019],"[509, 10, 783, 550, 346, 346, 604, 745, 357, 7..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[954, 525, 804, 834, 386, 452, 273, 0, 0, 145,...",[],"[954, 525, 804, 834, 386, 452, 273, 0, 0, 145,..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[523, 370, 935, 578, 493, 630, 322, 253, 293, ...",[],"[523, 370, 935, 578, 493, 630, 322, 253, 293, ..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[451, 668, 165, 55, 396, 833, 53, 782, 510, 13...",[],"[451, 668, 165, 55, 396, 833, 53, 782, 510, 13..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[954, 236, 90, 736, 797, 156, 278, 648, 736, 2...",[1058],"[954, 236, 90, 736, 797, 156, 278, 648, 736, 2..."
...,...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,"[989, 829, 829, 829, 738, 585, 604, 284, 284, ...",[],"[989, 829, 829, 829, 738, 585, 604, 284, 284, ..."
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,"[20, 677, 517, 242, 20, 20, 517, 517, 575, 517...",[1014],"[20, 677, 517, 242, 20, 20, 517, 517, 575, 517..."
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,"[164, 108, 482, 937, 856, 937, 937, 235, 133, ...",[],"[164, 108, 482, 937, 856, 937, 937, 235, 133, ..."
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,"[731, 197, 758, 533, 731, 282, 103, 731, 731, ...",[],"[731, 197, 758, 533, 731, 282, 103, 731, 731, ..."


In [30]:
# cluster별 빈도별로 sorting

song_freq = pd.DataFrame(pd.Series(chain.from_iterable(tr.songs)).value_counts(), columns = ["freq"])
song_freq['cid'] = [sid_to_cid[str(i)] for i in song_freq.index.tolist()]
song_freq_dict = song_freq.sort_values(['cid','freq'], ascending=False).groupby('cid').apply(lambda x:x.index.tolist())
song_freq_dict

cid
0      [677410, 696302, 552458, 19022, 98058, 109237,...
1      [400365, 487049, 425869, 291096, 559596, 13501...
10     [418438, 639570, 664126, 216504, 515252, 34496...
100    [6636, 422089, 150877, 295217, 499529, 220449,...
101    [98390, 92362, 212527, 300260, 4849, 118409, 1...
                             ...                        
995    [368205, 293558, 487742, 390868, 474196, 51960...
996    [294469, 595778, 628685, 692141, 118727, 14485...
997    [296599, 401892, 216879, 284390, 498317, 69062...
998    [109219, 198549, 575769, 345834, 518930, 55636...
999    [210289, 645786, 99950, 132887, 4030, 14941, 4...
Length: 1000, dtype: object

In [9]:
# prediction에 이용할 모델

algo = pd.read_pickle("/content/drive/My Drive/recommendation sys/0802/algo_0802.pkl")
train_obj = pd.read_pickle("/content/drive/My Drive/recommendation sys/0802/train_obj_0802.pkl")
user_matrix = algo.pu
item_matrix = algo.qi
raw_to_inner = {int(train_obj.to_raw_iid(i)):i for i in range(n_clusters+n_tags)}

# Prediction (detail)
> Prediction 과정을 자세히 설명합니다.

In [14]:
sorted_idx = val.tid.map(len).sort_values().index.tolist()

In [15]:
val.iloc[sorted_idx].tail(50)

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
14294,"[댄스, 드라이브, 힐링, 기분전환, 노래방]",49516,,"[274165, 416695, 103192, 548896, 79321, 516126...",4,2020-03-14 05:17:57.000,"[424, 425, 650, 194, 851, 243, 380, 145, 655, ...","[1058, 1001, 1022, 1007, 1098]","[424, 425, 650, 194, 851, 243, 380, 145, 655, ..."
17314,"[매장음악, 연주곡, 피아노, 힐링, 뉴에이지]",123897,우울한 나에게 건내는 위로와 같은 음악:),[],1,2020-02-24 07:34:25.000,[],"[1011, 1045, 1108, 1022, 1017]","[1011, 1045, 1108, 1022, 1017]"
3721,"[감성, 비, 카페, 기분전환, 이별]",121077,,"[236616, 169694, 462002, 412676, 340538, 26682...",6,2017-08-12 06:27:03.000,"[372, 766, 97, 580, 56, 97, 493, 935, 56, 97, ...","[1004, 1061, 1033, 1007, 1047]","[372, 766, 97, 580, 56, 97, 493, 935, 56, 97, ..."
6482,"[카페, 커피, 가을, 따뜻한, 밤]",21911,,"[346017, 658963, 147889, 659791, 276165, 37465...",17,2019-12-26 21:23:34.000,"[176, 976, 744, 369, 53, 53, 53, 967, 53, 448,...","[1033, 1096, 1067, 1062, 1027]","[176, 976, 744, 369, 53, 53, 53, 967, 53, 448,..."
12418,"[우울, 새벽, 위로, 힐링, 이별]",73414,,"[364040, 549392, 512994, 643628, 469799, 11215...",7,2020-04-14 23:02:13.000,"[56, 804, 507, 804, 133, 804, 57, 983, 804, 14...","[1054, 1034, 1025, 1022, 1047]","[56, 804, 507, 804, 133, 804, 57, 983, 804, 14..."
16721,"[겨울, 감성, 눈물, 발라드, 커피]",127820,,"[42036, 142584, 683053, 653618, 553705, 186284...",2,2019-12-30 00:04:32.000,"[890, 485, 626, 804, 626, 485, 890, 485, 365, ...","[1050, 1004, 1069, 1020, 1096]","[890, 485, 626, 804, 626, 485, 890, 485, 365, ..."
20650,"[밤, 우울, 새벽, 드라이브, 퇴근길]",72508,,"[549966, 188349, 363429, 257277, 481669, 37748...",13,2017-11-03 10:27:12.000,"[798, 626, 798, 386, 798, 804, 804, 804, 804, ...","[1027, 1054, 1034, 1001, 1101]","[798, 626, 798, 386, 798, 804, 804, 804, 804, ..."
16718,"[밤, 오후, 추억, 위로, 새벽]",56972,,"[680366, 476773, 77951, 389920, 683520, 410940...",1,2019-11-20 05:27:23.000,"[485, 680, 804, 145, 857, 56, 273, 273, 485, 2...","[1027, 1043, 1002, 1025, 1034]","[485, 680, 804, 145, 857, 56, 273, 273, 485, 2..."
19544,"[알앤비, 힐링, 기분전환, 팝, 어쿠스틱]",26202,,"[128473, 401209, 636601, 79670, 72891, 60078, ...",15,2019-03-17 12:24:27.000,"[383, 755, 282, 536, 218, 692, 591, 504, 730]","[1031, 1022, 1007, 1008, 1065]","[383, 755, 282, 536, 218, 692, 591, 504, 730, ..."
2603,"[봄, 연인, 산책, 기분전환, 사랑]",116344,,"[461680, 98223, 293236, 650494, 204547, 22643,...",9,2018-04-03 09:36:51.000,"[626, 968, 626, 804, 680, 625, 510, 804, 680, ...","[1075, 1063, 1024, 1007, 1046]","[626, 968, 626, 804, 680, 625, 510, 804, 680, ..."


In [61]:
############# 1. SVD 행렬을 행 별로 normalize 시킨다.

n_values = n_clusters + n_tags
org = user_matrix.dot(item_matrix.T) # org_matrix
norm = np.linalg.norm(org,axis=1).reshape(-1,1)
SVD_mat_scaled = org/norm


# 결과를 확인하고 싶은 index 지정
# idx = 8086
idx = 1046 # 배경음악, 카페, 독서, 뉴에이지 등
# idx = 7702
target = val.iloc[idx]

# extract clusters/tags 
values = [train_obj.to_inner_iid(str(i)) for i in val.iloc[idx].clusters_tags]

# convert to one-hot vector
s = np.eye(n_values)[values].sum(axis=0)

# calcuate similarity
sim = SVD_mat_scaled.dot(s)
most_similar_ply = sim.argmax() # the most similar plylst

display(val.iloc[[idx]])

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
1046,"[배경음악, 카페, 독서, 뉴에이지, 아침]",102119,,"[439336, 335116, 552555, 17328, 29793, 113724,...",2,2020-01-13 14:24:31.000,"[70, 133, 302, 113, 133, 52, 302, 855, 302, 76...","[1033, 1042, 1017, 1048]","[70, 133, 302, 113, 133, 52, 302, 855, 302, 76..."


In [62]:
# 위에서 추출한 가장 유사한 plylst 의 실제 값 확인해보기
tr.query(f'id=={train_obj.to_raw_uid(most_similar_ply)}')

,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
60166,"[카페, 연주곡, 매장, bgm, 독서, 뉴에이지, 가을, 공부, 밤]",137903,가을날 카페에서 틀어주는 피아노 BGM,"[304995, 131690, 645171, 622433, 251746, 64691...",12,2019-01-29 19:13:13.000,"[214, 214, 214, 457, 855, 821, 214, 821, 302, ...",[],"[214, 214, 214, 457, 855, 821, 214, 821, 302, ..."


In [63]:
#########
# songs #
#########

song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
sim_clusters = SVD_mat_scaled[most_similar_ply,song_inner_id] # 가장 유사한 유저의 노래 점수를 추출

songs = set(target.songs)
len_ = len(songs)

n = 100
end_num = len_ + n

# 점수를 고려하여 cluster별로 노래를 선택합니다.
# ex : cluster 1에 대한 점수가 0.3인 경우 : cluster 1에서 30개의 노래를 빈도 순으로 추출한다.

for ratio, cluster_inner_id in sorted(zip(sim_clusters,song_inner_id), reverse=True):
    cluster_id = int(train_obj.to_raw_iid(cluster_inner_id))
    display(song_meta.query(f'new_song_id == {cluster_id}').head())
    num = int(n*ratio) + 1
    limit =len_ + num

    for song in song_freq_dict.loc[str(cluster_id)]:
        end_state = (len(songs) >= end_num)
        if (len(songs) >= limit) or end_state:
            break
        songs.add(str(song))
    
    if end_state:
        break
    len_ = len(songs)

recommended_songs = songs - set(target.songs)

print("Ground truth :", [sid_to_song[i] for i in target.songs])
print("Recommended songs :")
for i in recommended_songs:
    print(sid_to_song[int(i)])

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
580,"[GN1806, GN1801]",20161128,겨울밤,10018484,[944046],겨울밤,[GN1800],[프링 (Pring)],580,"[카페, 잔잔한, 커피, 기분전환, 피아노, 새벽, 뉴에이지, 자장가, 밤, 아침,...",821
3603,[GN1801],20170120,밤노을,10032740,[713816],밤노을,[GN1800],[루바토],3603,"[피아노, 연주곡, 집중, 독서, 공부, 감성, 주말, 잔잔한, 오후, 휴식, 가을...",821
4034,"[GN1806, GN1801]",20150414,Happy Spring Day,2313730,[713917],우리 동네에 햇살이 내리면,[GN1800],[러브래빗],4034,"[여유, 연주곡, 피아노, 독서, 카페, 자장가, 집중, 잠들기전, 공부, 뉴에이지...",821
5724,[GN1801],20121224,The Place Dancing With Snow,2170846,[695258],찻잔과 도넛이 춤추는 가게 (Christmas Ver.),[GN1800],[세레노],5724,"[휴식, 밤, 잔잔한, 새벽, 감성, 뉴에이지, 피아노, 새벽감성, 겨울, 까페, ...",821
6868,"[GN1806, GN1801]",20160516,너의 색으로 물들다,2684910,[476012],너의 색으로 물들다,[GN1800],[신지호],6868,"[뉴에이지, 집중, 자장가, 공부, 가을, 피아노, 커피, 독서, 비오는날, 힐링,...",821


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
579,"[GN1806, GN1801]",20170516,f l o w e r,10062379,[8048],f l o w e r,[GN1800],[이루마],579,"[겨울, 휴식, 아침, 연주곡, 오후, 주말, 위로, 뉴에이지, 가을, 잔잔한, 피...",302
1867,"[GN1806, GN1801]",20150210,우리 동네에 햇살이 내리면,2303885,[713917],우리 동네에 햇살이 내리면,[GN1800],[러브래빗],1867,"[스트레스, 연주곡, 여름, 잔잔한, 피아노, OST, 뉴에이지, 비오는날, 감성,...",302
6945,"[GN1806, GN1801]",20161031,위로가 필요한 밤에,10010784,[1227992],위로가 필요한 밤에,[GN1800],[Little Fish],6945,"[감성, 매장음악, 연주곡, 가을, 새벽, 피아노, 커피, 뉴에이지, 위로, 힐링,...",302
10532,"[GN1805, GN1803, GN1801]",20060419,Ciao! Bravo!!,325044,[172765],Kitto Mata Itsuka (분명 다시 언젠가),[GN1800],[Depapepe],10532,"[연주곡, 카페, 피아노, 힐링, 기분전환, 뉴에이지, 스트레스, 휴식, 노동요, ...",302
12939,"[GN1503, GN1501]",19940000,러브 어페어 OST,27359,[2988],Piano Solo,[GN1500],[Ennio Morricone],12939,"[연주곡, 카페, 피아노, 힐링, 기분전환, 뉴에이지, 스트레스, 휴식, 잔잔한, ...",302


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
820,"[GN1802, GN1801]",20110706,영화속 주옥 같은 OST 테마음악,1343461,[554320],Romance (금지된장난),[GN1800],[Trevor Nasser],820,"[카페, 잔잔한, 오후, 피아노, 뉴에이지, 잠들기전, 자장가, 감성, 연주곡, 휴...",214
950,[GN1801],20181222,"명상, 요가, 숙면, 불면증, 우울증 치료에 좋은 뮤직테라피 뉴에이지 피아노 ASM...",10235953,[2203026],별과 별 사이에 (잔잔한 빗소리),[GN1800],[우리동네피아노],950,"[연주곡, 잔잔한, 집중, 공부, 밤, 피아노, 뉴에이지, 힐링, 새벽, 휴식, 매...",214
1518,"[GN1806, GN1801]",20160930,피아노,10002690,[721438],새벽 편지,[GN1800],[피아노],1518,"[휴식, 혼자, 잔잔한, 위로, 힐링, 뉴에이지, 감성, 아침, 출근길, 기분전환,...",214
2434,[GN1801],20180112,La Granada,10127814,[677652],La Granada (Piano Ver.),[GN1800],[최병욱],2434,"[힐링, 뉴에이지, 주말, 연주곡, 스트레스, 기분전환, 집중, 위로, 잠들기전, ...",214
2931,[GN1801],20171206,힐링을 위한 감성 피아노 연주곡 베스트,10117047,[2019643],"바다, 그리고 기억 (Sea Ver.)",[GN1800],[명상음악],2931,"[피아노, 힐링, 집중, 노동요, 카페, 연주곡, 공부, 휴식, 위로, 감성, 자장...",214


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
110,"[GN1403, GN1401]",20180907,20th Century Masters: The Millennium Collectio...,5643752,[30262],Wayfaring Pilgrim,[GN1400],[Roy Buchanan],110,"[록, 기분전환, Rock, 락, 신나는]",362
291,"[GN1402, GN1401]",20010424,Natural Woman The Very Best Of Carole King,907,[12039],Up On The Roof (Live),[GN1400],[Carole King],291,[락],362
620,"[GN1403, GN1401]",20050927,Bring `Em In,310690,[27343],Cheaper To Keep Her/Blues In The Night,[GN1400],[Buddy Guy],620,"[감성, 락]",362
1028,"[GN1402, GN1401, GN1405]",19710000,Blue,26802,[26645],Little Green,[GN1400],[Joni Mitchell],1028,"[감성, 기분전환, 밤, 락]",362
1030,"[GN1402, GN1401]",20010821,"Parsley, Sage, Rosemary And Thyme (Bonus Track)",338140,[102837],The 59Th Street Bridge Song (Feelin` Groovy),[GN1400],[Simon & Garfunkel],1030,[락],362


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags,new_song_id
683,[GN1801],20190319,The End Of The World,10262305,[2635414],When You Feel Love,[GN1800],[TSUUDEN YOSHIZO],683,"[출근길, 아침, 새벽, 위로, 잠들기전, 퇴근길, 밤, 휴식, 연주곡, 오후, 피...",190
1552,"[GN2806, GN2801]",20190315,사랑스러운 반려동물을 위한 명품 피아노 모음집,10261184,[2634990],나비야 (바닷가 Ver.),[GN2800],[강아지 음악],1552,"[집중, 연주곡, 피아노, 기분전환, 공부, 노동요, 오후, 독서, 뉴에이지, 잠들...",190
2996,[GN1801],20180122,포근하고 편안한 숙면을 위한 잔잔하고 따뜻한 감성 피아노 연주곡 베스트,10130962,[2062502],달콤한 잠 (Sweet Dream),[GN1800],[자장가],2996,"[연주곡, 피아노, 자장가, 편안한, 집중, 오후, 뉴에이지, 잠들기전, 휴식, 노...",190
3768,[GN1801],20180102,마음의 평온을 위한 감성 명상 힐링 피아노 연주곡 베스트,10124107,[2059910],우주의 소리 (aum),[GN1800],[요가음악],3768,"[아침, 스트레스, 자장가, 운동, 힐링, 뉴에이지, 노동요, 피아노, 공부, 집중...",190
5377,"[GN1701, GN1703]",20150630,로맨틱 샘물고기,2326625,[873263],이상한 꿈 (String Ensembles With Piano),[GN1700],[이샘물],5377,"[여유, 힐링, 편안한, 뉴에이지, 봄, 카페, 휴식, 연주곡, 피아노, 잠들기전,...",190


Ground truth : ['Danny Boy', 'Merry Christmas Mr. Lawrence', 'Reminiscent (회상)', '그럴 수 있다면', 'River Flows In You', '다시 돌아온 계절', 'Chaconne', '함께 걷던 길', 'Suddenly', '낯선 하루 (A Strange Day)', '오늘은 왠지...', '이제는 쉬어도 돼', '아픔을 잊는 법', '그대를 생각했던 순간들', '내리던 가을잎은', '그 집', '환상(illusion fantasy)', '위로가 필요한 밤', '놀이터', '기도', '마음정원']
Recommended songs :
Cafe Flore
Daily Life
가장 조용했던 밤의 위로
Little Flowers
별의 바다 (Sea Of Stars)
Spring Breeze Waltz
내리던 가을잎은
Ciels d`hiver (겨울 풍경)
명상음악 (조용한 사색)
The Moment
자장가
항상 그곳에
부탁해요
마음정원
Always Happy
위로
Lie To Me
오늘은 왠지...
Blue Sea
울면 안돼
Chopin : Waltz No.9 In A Flat Major Op.69 `L`Adieu` (쇼팽 : 왈츠 9번 내림 가장조 작품번호 69 `이별의 왈츠`)
사랑의 인사
사랑하는 이들에게
사랑극
Lost Memory
Dreaming Of April (멋진 하루)
Passing By
녹턴
고양이의 하루
쉬고 싶은 나무
Happy And Sad
Day Is Done
The Whistler`s Song
Suddenly
May Be
Like A Rolling Stone
Smile Smile Smile
What Kind Of Woman Is This
이제는 안녕
Silent moon
The Shape Of A Woman
Happy Day
Before The Next Teardrop Falls
Linda 행진곡
밤하늘에 쓰는 편지
그랬던 밤처럼
They Bring Me To You (Fe

In [64]:
########
# tags #
########

tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)])
sim_tags = SVD_mat_scaled[most_similar_ply,tag_inner_id]

recommended_tags = set(target.tags)
len_ = 

for _, tag_id in sorted(zip(sim_tags, tag_inner_id), reverse=True):
    recommended_tags.add(train_obj.to_raw_iid(tag_id))
    if len(recommended_tags) == 10 + :
        break

recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

print("Ground truth :", target.tags)
print("Recommended tags :",recommended_tags)

Ground truth : ['배경음악', '카페', '독서', '뉴에이지', '아침']
Recommended tags : ['편안한', '따뜻한', '저녁', '매장', '독서']


# Prediction



In [ ]:
from tqdm import tqdm

In [ ]:
answers = []

In [ ]:
n_values = n_clusters + n_tags
org = user_matrix.dot(item_matrix.T) # org_matrix
norm = np.linalg.norm(org,axis=1).reshape(-1,1)
SVD_mat_scaled = org/norm

In [ ]:
for idx in range(len(val)):

    target = val.iloc[idx]

    # extract clusters/tags
    values = [train_obj.to_inner_iid(str(i)) for i in target.clusters_tags]

    # convert to one-hot vector
    s = np.eye(n_values)[values].sum(axis=0)
    sim = org.dot(s) # sim.argmax() ; 가장 유사한 cluster"

    song_inner_id = [raw_to_inner[i] for i in range(0,1000)]
    sim_cluters = SVD_mat_scaled[sim.argmax(),song_inner_id]

    songs = set(target.songs)
    len_ = len(songs)

    n = 100
    end_num = len_ + n

    for ratio, cluster_id in zip(np.sort(sim_cluters)[::-1], np.argsort(sim_cluters)[::-1]):
        num = int(n*ratio) + 1
        limit =len_ + num

        for song in cluster[cluster_id]:
            end_state = (len(songs) >= end_num)
            if (len(songs) >= limit) or end_state:
                break
            songs.add(str(song))
        
        if end_state:
            break
        len_ = len(songs)

    recommended_songs = songs - set(target.songs)

    tag_inner_id = np.array([raw_to_inner[i] for i in range(1000,1110)]) # 1032,1068,1101
    sim_tags = np.argsort(SVD_mat_scaled[sim.argmax(),tag_inner_id])[::-1]

    recommended_tags = set(target.tags)
    len_ = len(recommended_tags)

    n = 10
    end_num = len_ + n

    for i in tag_inner_id[sim_tags]: # 유사도별 내림차순
        recommended_tags.add(train_obj.to_raw_iid(i))
        if len(recommended_tags) == end_num:
            break

    recommended_tags = [tid_to_tag[i] for i in recommended_tags - set(target.tags)]

    answers.append({
    "id": target.id,
    "songs": list(recommended_songs),
    "tags": list(recommended_tags)
    })

In [ ]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [ ]:
# tag mapping 안됐을 때

a = pd.DataFrame(answers)
a['tags'] = a.tags.map(lambda x: [tid_to_tag[t] for t in x])
a['tags'] = a.tags.map(lambda x: [int(t) for t in x])
a['songs'] = a['songs'].map(lambda x: [int(s) for s in x])
res = list(a.transpose().to_dict().values())

In [ ]:
write_json(answers,"/content/drive/My Drive/recommendation sys/results_0802_split.json")